# Mottonen State Preparation
Elias A. Lehman | B.A. Physics, 2024 | University of California, Berkeley

## Introduction

A problem arises when the theory of quantum gate operations diverges from the physical limitations of quatum computers. Particularly, the set of gate operations used in common practice is larger that the set of gates we are able to implement with decent fidelity, if at all. As a solution, it is often required that physcial gate operations applied to qubits are the fundamentally decomposed equivalent of more complex operations.

Assuming that a contemporary quantum computer is capable of the *elementary set* of gates, consisting of the C-NOT gate and one-qubit rotation gates minus the phase gate, it can be shown that an arbitrary state preparation can be achieved by a combination of the this set. Specifically, the literature [1] presents a method which utilizes uniformly controlled rotations to transform a quantum circuit into it's elementary equivalent requiring at most $2^n+2-4n-4$ CNOT gates and $2^{n+2}-5$ one-qubit rotations, a significant improvement from the previously presented general case requiring $O(n2^n)$ gates.




In [2]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 44.6 MB/s 
     |████████████████████████████████| 19.2 MB 1.1 MB/s 
     |████████████████████████████████| 240 kB 78.5 MB/s 
     |████████████████████████████████| 112 kB 77.0 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 943 kB 102.3 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 4.0 MB 66.4 MB/s 
     |████████████████████████████████| 1.9 MB 68.8 MB/s 
     |████████████████████████████████| 112 kB 76.8 MB/s 
  Created wheel for qiskit: filename=qiskit-0.38.0-py3-none-any.whl size=12147 sha256=694d4831abb2828e219c4807166054d03cf480e4b50259ac4ed0d622b2964dad
  Stored in directory: /root/.cache/pip/wheels/0b/30/78/04c

In [58]:
#A library to assist with applying Uniform Control Rotations using SciPy sparse.dok_matrix objects
!pip install dc_qiskit_algorithms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for nxpd: filename=nxpd-0.2.0-py3-none-any.whl size=28315 sha256=216a370efdec9338b9286fcc5783c5dd8b0ed97f03e60c4b4ae5ea9d9fac26cc
  Stored in directory: /root/.cache/pip/wheels/43/cc/3c/19624f0b4ee689a58de001acc59e17daef1d53400d54ee3b27
Successfully built nxpd


In [3]:
#We import the following packages
import math
import numpy as np
from qiskit import *

#During computation all vecotrs and matrices are treated using this library for effieciency
from scipy import sparse

In [155]:
from qiskit.converters.ast_to_dag import RZGate
from qiskit.extensions.quantum_initializer.ucrz import ucrz
from qiskit.extensions.quantum_initializer.ucry import ucry
from qiskit.circuit.library.arithmetic.exact_reciprocal import UCRYGate
from dc_qiskit_algorithms.UniformRotation import *
#We define two functions for calculating the nessesary alphas of the Z and Y rotations on a certain qubit, k:
def getAlphaZ(n, k, omega):
  """ Calculate alphas for Z rotation.  
 
    Parameters
    ------------
        n: number of qubits
        k: current qubit
        omega: input vector phase

    Returns
    -----------
      alphaZk: vector containing respective alpha values
  """
  alphaZk = sparse.dok_matrix((2**(n-k), 1))
  for (i,_), w in omega.items():
        i += 1
        j = int(np.ceil(i * 2 ** (-k)))
        s_i = 1.0 if i > 2 ** (k - 1) * (2 * j - 1) else -1.0
        alphaZk[j - 1, 0] = alphaZk[j - 1, 0] + s_i * w / 2 ** (k - 1)

  return alphaZk

def getAlphaY(n, k, A):
  """ Calculate alphas for Y rotation.  
 
    Parameters
    ------------
        n: number of qubits
        k: current qubit
        A: vector of input magnitudes

    Returns
    -----------
      alphaY: vector containing respective alpha values
  """
  alpha = sparse.dok_matrix((2**(n - k), 1))

  numerator = sparse.dok_matrix((2 ** (n - k), 1))
  denominator = sparse.dok_matrix((2 ** (n - k), 1))

  for (i,_), a in A.items():
      j = int(math.ceil((i + 1) * 2** (-k)))
      l = (i + 1) - (2*j - 1)*2**(k-1)

      #For l between 1 and 2^(k-1)...
      is_numerator = 1 <= l <= 2**(k-1)

      #...add the square of this magnitude to the sum in the numerator.
      if is_numerator:
          numerator[j - 1, 0] += a*a
      #Add all magnitudes (from l=1 -> l=2^k) to the sum in the denominator.
      denominator[j - 1, 0] += a*a

  #For all sums in the numerator and denominator vectors, take the square root.
  for (j, _), a in numerator.items():
      numerator[j, 0] = np.sqrt(a)
  for (j, _), a in denominator.items():
      denominator[j, 0] = 1/np.sqrt(a)

  
  arcsin_angle = numerator.multiply(denominator)
  for (j, _), a in arcsin_angle.todok().items():
      alpha[j, 0] = 2*np.arcsin(a)

  return alpha


#Next, we define a function for applying the uniformly controlled Y and Z rotation cascades.

def XiY(qreg, V):
  """ Applies cascade of Y rotations to given quantum register.  
 
    Parameters
    ------------
        qr: QuantumRegister onto which the gate applies
        V: vector of input magnitudes or angles

    Returns
    -----------
      Xi: a quantum circuit composed of the cascading Y rotations
  """
  num_qubits = int(math.log2(V.shape[0]))

  Xi = QuantumCircuit(qreg, name='XiY')
  
  for k in range(1, num_qubits + 1):
    alphas = getAlphaY(num_qubits, k, V)
    control = qreg[k:]
    target = qreg[k - 1]
    Xi.append(UniformRotationGate(RYGate, alphas), control + [target], [])
  return Xi


def XiZ(qreg, V):
  """ Applies cascade Z rotations to given quantum register.  
 
    Parameters
    ------------
        qr: QuantumRegister onto which the gate applies
        V: vector of input magnitudes or angles

    Returns
    -----------
      Xi: a quantum circuit composed of the cascading Z rotations
  """
  num_qubits = int(math.log2(V.shape[0]))

  Xi = QuantumCircuit(qreg, name='XiZ')
  
  for k in range(1, num_qubits + 1):
    alphas = getAlphaY(num_qubits, k, V)
    control = qreg[k:]
    target = qreg[k - 1]
    Xi.append(UniformRotationGate(RZGate, alphas), control + [target], [])

  return Xi


def möttönen(self, a, qubits):
    # type: (QuantumCircuit, Union[List[float], sparse.dok_matrix], Union[List[Qubit], QuantumRegister]) -> Instruction
  """ Applies Möttönen State Preparation.  
 
    Parameters
    ------------
        self: Quantum Circuit on which to apply preparation
        a: Statevector to achieve
        qubits: list of qubits on which to apply the preperation

    Returns
    -----------
      self: prepared Quantum Circuit
  """
  if isinstance(qubits, QuantumRegister):
        qubits = list(qubits)

  if isinstance(a, sparse.dok_matrix):
    num_qubits = int(math.log2(a.shape[0]))
    self.state = a
    A = sparse.dok_matrix(self.state.get_shape())  # type: sparse.dok_matrix
    omega = sparse.dok_matrix(self.state.get_shape())  # type: sparse.dok_matrix

    for (i, j), v in self.state.items():
      A[i, j] = np.absolute(v)
      omega[i, j] = np.angle(v)

      q = QuantumRegister(self.num_qubits, "qubits")

      Y_rotation = XiY(q, A)
      self.append(Y_rotation.inverse(), range(self.num_qubits))

      Z_rotation = XiZ(q, omega)
      self.append(Z_rotation.inverse(), range(self.num_qubits))
    return self
  else:
    return möttönen(self, sparse.dok_matrix([a]).transpose(), qubits)

QuantumCircuit.möttönen = möttönen

In [156]:
qr = QuantumRegister(2, 'q')
qc = QuantumCircuit(qr)

state = [0, 1/np.sqrt(2), 1/np.sqrt(2), 0]
qc.möttönen(state, qr)
print("Möttönen State decomposed into Cascades:")
qc = qc.decompose()
print(qc.draw())
print("\n\nCascades decomposed to fundamental gates:\n")
print(qc.decompose().draw())

simulator = Aer.get_backend('statevector_simulator')
result = execute(qc, simulator).result()
statevector = result.get_statevector(qc)
print(statevector)

Möttönen State decomposed into Cascades:
                      ┌────────────────┐                 ┌────────────────┐»
q_0: ─────────────────┤1               ├─────────────────┤1               ├»
     ┌───────────────┐│  uni_rot_ry_dg │┌───────────────┐│  uni_rot_rz_dg │»
q_1: ┤ uni_rot_ry_dg ├┤0               ├┤ uni_rot_rz_dg ├┤0               ├»
     └───────────────┘└────────────────┘└───────────────┘└────────────────┘»
«                      ┌────────────────┐                 ┌────────────────┐
«q_0: ─────────────────┤1               ├─────────────────┤1               ├
«     ┌───────────────┐│  uni_rot_ry_dg │┌───────────────┐│  uni_rot_rz_dg │
«q_1: ┤ uni_rot_ry_dg ├┤0               ├┤ uni_rot_rz_dg ├┤0               ├
«     └───────────────┘└────────────────┘└───────────────┘└────────────────┘


Cascades decomposed to fundamental gates:

              ┌───┐┌──────────┐┌───┐┌──────────┐┌───┐┌───────┐┌───┐ ┌───────┐  »
q_0: ─────────┤ X ├┤ Ry(-π/2) ├┤ X ├┤ Ry(-π/2) ├┤ X ├┤ Rz(0) ├┤